# NumerAI Colab Training Runner

This notebook is intentionally thin: it clones the repo, installs dependencies, and runs `src/train_colab.py`.

- Edit code on GitHub (or locally), not in Colab.
- Use Colab as a disposable execution environment only.


In [ ]:
import os
from pathlib import Path

REPO_URL = os.getenv("REPO_URL", "https://github.com/<your-org-or-user>/Numerai-Re.git")
REPO_REF = os.getenv("REPO_REF", "")  # optional: branch/tag/commit SHA
REPO_DIR = Path(os.getenv("REPO_DIR", "/content/Numerai-Re"))

try:
    from google.colab import userdata
except Exception:
    userdata = None

if userdata and "github.com" in REPO_URL and "GH_TOKEN" not in REPO_URL:
    gh_token = userdata.get("GH_TOKEN")
    if gh_token:
        REPO_URL = REPO_URL.replace("https://", f"https://{gh_token}@")

os.environ["REPO_URL"] = REPO_URL
os.environ["REPO_REF"] = REPO_REF
os.environ["REPO_DIR"] = str(REPO_DIR)

masked_repo_url = REPO_URL
if masked_repo_url.startswith("https://") and "github.com" in masked_repo_url and "@" in masked_repo_url:
    # Mask any credentials embedded between 'https://' and '@' to avoid leaking tokens in logs
    prefix, rest = masked_repo_url.split("https://", 1)
    creds_and_host = rest.split("@", 1)
    if len(creds_and_host) == 2:
        _, host_and_path = creds_and_host
        masked_repo_url = "https://***@" + host_and_path

print(f"REPO_URL={masked_repo_url}")
print(f"REPO_REF={REPO_REF or '(default branch)'}")
print(f"REPO_DIR={REPO_DIR}")


In [ ]:
if not REPO_DIR.exists():
    !git clone "$REPO_URL" "$REPO_DIR"

!bash "$REPO_DIR/scripts/colab_bootstrap.sh"


In [ ]:
%cd $REPO_DIR
!python src/train_colab.py
